Requirements:

Your task is to implement a simplified version of an in-memory database. Plan your design according
to the level specifications below:
- Level 1: In-memory database should support basic operations to manipulate records,
fields, and values within fields.
- Level 2: In-memory database should support displaying a specific record's fields based on a
filter.
- Level 3: In-memory database should support TTL (Time-To-Live) configurations on database
records.
- Level 4: In-memory database should support backup and restore functionality.
To move to the next level, you need to pass all the tests at this level.

Note:

You will receive a list of queries to the system, and the final output should be an array of strings
representing the returned values of all queries. Each query will only call one operation.

Level 1

The basic level of the in-memory database contains records. Each record can be accessed with a
unique identifier key of string type. A record may contain several field-value pairs, both of which
are of string type.

- `SET <key> <field> <value>` — should insert a field-value pair to the record associated
with key. If the field in the record already exists, replace the existing value with the
specified value. If the record does not exist, create a new one. This operation should return
an empty string.

- `GET <key> <field>` — should return the value contained within field of the record
associated with key. If the record or the field doesn't exist, should return an empty string.

- `DELETE <key> <field>` — should remove the field from the record associated with key.
Returns "true" if the field was successfully deleted, and "false" if the key or the field do
not exist in the database.


Examples

The example below shows how these operations should work (the section is scrollable to the right):

| Queries | Explanations |
|---------|--------------|
| `["SET", "A", "B", "E"]` | returns ""; database state: `{"A": {"B": "E"}}` |
| `["SET", "A", "C", "F"]` | returns ""; database state: `{"A": {"C": "F", "B": "E"}}` |
| `["GET", "A", "B"]` | returns "E" |
| `["GET", "A", "D"]` | returns "" |
| `["DELETE", "A", "B"]` | returns "true"; database state: `{"A": {"C": "F"}}` |
| `["DELETE", "A", "D"]` | returns "false"; database state: `{"A": {"C": "F"}}` |

Above table is just to show matching query and result one-by-one.

Note: You will recieve a list of queries to the system:

`queries = [
["SET", "A", "B", "E"],
["SET", "A", "C", "F"],
["GET", "A", "B"],
["GET", "A", "D"],
["DELETE", "A", "B"],
["DELETE", "A", "D"]
]`

And the final output should be `["", "", "E", "", "true", "false"]`

In [29]:
import copy

class MemDB:
    def __init__(self):
        self.data = {}

    def set(self, key, field, value):
        if key not in self.data:
            self.data[key] = {}
        self.data[key][field] = value
        return ""

    def get(self, key, field):
        if key in self.data:
            return self.data[key].get(field, "")
        return ""
    
    def delete(self, key, field):
        if key in self.data and field in self.data[key]:
            del self.data[key][field]
            return "true"
        return "false"
    

def solution(queries):
    db = MemDB()
    res = []
    db_status = []
    for query in queries:
        if query[0] == "SET":
            res.append(db.set(query[1], query[2], query[3]))
        elif query[0] == "GET":
            res.append(db.get(query[1], query[2]))
        elif query[0] == "DELETE":
            res.append(db.delete(query[1], query[2]))

        db_status.append(copy.deepcopy(db.data))

    return res, db_status

queries = [
    ["SET", "A", "B", "E"],
    ["SET", "A", "C", "F"],
    ["GET", "A", "B"],
    ["GET", "A", "D"],
    ["DELETE", "A", "B"],
    ["DELETE", "A", "D"]
]

res, db_status = solution(queries)
print(res)
print(db_status)


['', '', 'E', '', 'true', 'false']
[{'A': {'B': 'E'}}, {'A': {'B': 'E', 'C': 'F'}}, {'A': {'B': 'E', 'C': 'F'}}, {'A': {'B': 'E', 'C': 'F'}}, {'A': {'C': 'F'}}, {'A': {'C': 'F'}}]


Level 2

The database should support displaying data based on filters. Introduce an operation to support
printing some fields of a record.

- `SCAN <key>` — should return a string representing the fields of a record associated with key.
The returned string should be in the following format "`<field1>(<value1>),
<field2>(<value2>), ...`", where fields are sorted lexicographically. If the specified record
does not exist, returns an empty string.

- `SCAN_BY_PREFIX <key> <prefix>` — should return a string representing some fields of a
record associated with key. Specifically, only fields that start with prefix should be included.
The returned string should be in the same format as in the SCAN operation with fields sorted
in lexicographical order.

Examples

The example below shows how these operations should work (the section is scrollable to the right):

| Queries | Explanations |
|---------|--------------|
| `["SET", "A", "BC", "E"]` | returns `""` <br>database state: `{"A": {"BC": "E"}}` |
| `["SET", "A", "BD", "F"]` | returns `""` <br>database state: `{"A": {"BC": "E", "BD": "F"}}` |
| `["SET", "A", "C", "G"]` | returns `""` <br>database state: `{"A": {"BC": "E", "BD": "F", "C": "G"}}` |
| `["SCAN_BY_PREFIX", "A", "B"]` | returns `"BC(E), BD(F)"` |
| `["SCAN", "A"]` | returns `"BC(E), BD(F), C(G)"` |
| `["SCAN_BY_PREFIX", "B", "B"]` | returns `""` |



Queries Explanations

`queries = [
    ["SET", "A", "BC", "E"],
    ["SET", "A", "BD", "F"],
    ["SET", "A", "C", "G"],
    ["SCAN_BY_PREFIX", "A", "B"],
    ["SCAN", "A"],
    ["SCAN_BY_PREFIX", "B", "B"]
]`

the output should be `["", "", "", "BC(E), BD(F)", "BC(E), BD(F), C(G)", ""]`.

In [30]:
import copy

class MemDB:
    def __init__(self):
        self.data = {}

    def set(self, key, field, value):
        if key not in self.data:
            self.data[key] = {}
        self.data[key][field] = value
        return ""

    def get(self, key, field):
        if key in self.data:
            return self.data[key].get(field, "")
        return ""
    
    def delete(self, key, field):
        if key in self.data and field in self.data[key]:
            del self.data[key][field]
            return "true"
        return "false"
    
    def scan(self, key): 
        if key in self.data:
            res = []
            for field, value in self.data[key].items():
                res.append("%s(%s)" % (field, value))
            return ", ".join(sorted(res))
        return ""

    def scan_by_prefix(self, key, prefix):
        if key in self.data:
            res = []
            for field, value in self.data[key].items():
                if field.startswith(prefix):
                    res.append("%s(%s)" % (field, value))
            return ", ".join(sorted(res))
        return ""
    

def solution(queries):
    db = MemDB()
    res = []
    db_status = []
    for query in queries:
        if query[0] == "SET":
            res.append(db.set(query[1], query[2], query[3]))
        elif query[0] == "GET":
            res.append(db.get(query[1], query[2]))
        elif query[0] == "DELETE":
            res.append(db.delete(query[1], query[2]))
        elif query[0] == "SCAN":
            res.append(db.scan(query[1]))
        elif query[0] == "SCAN_BY_PREFIX":
            res.append(db.scan_by_prefix(query[1], query[2]))

        db_status.append(copy.deepcopy(db.data))

    return res, db_status

queries = [
    ["SET", "A", "BC", "E"],
    ["SET", "A", "BD", "F"],
    ["SET", "A", "C", "G"],
    ["SCAN_BY_PREFIX", "A", "B"],
    ["SCAN", "A"],
    ["SCAN_BY_PREFIX", "B", "B"]
]

res, db_status = solution(queries)
print(res)
print(db_status)


['', '', '', 'BC(E), BD(F)', 'BC(E), BD(F), C(G)', '']
[{'A': {'BC': 'E'}}, {'A': {'BC': 'E', 'BD': 'F'}}, {'A': {'BC': 'E', 'BD': 'F', 'C': 'G'}}, {'A': {'BC': 'E', 'BD': 'F', 'C': 'G'}}, {'A': {'BC': 'E', 'BD': 'F', 'C': 'G'}}, {'A': {'BC': 'E', 'BD': 'F', 'C': 'G'}}]


Level 3

Support the timeline of operations and TTL (Time-To-Live) settings for records and fields. Each
operation from previous levels now has an alternative version with a timestamp parameter to
represent when the operation was executed. For each field-value pair in the database, the TTL
determines how long that value will persist before being removed.

Notes:
- Time should always flow forward, so timestamps are guaranteed to strictly increase as
operations are executed.

- Each test cannot contain both versions of operations (with and without timestamp). However,
you should maintain backward compatibility, so all previously defined methods should work
in the same way as before.

- `SET_AT <key> <field> <value> <timestamp>` — should insert a field-value pair or
updates the value of the field in the record associated with key. This operation should
return an empty string.

- `SET_AT_WITH_TTL <key> <field> <value> <timestamp> <ttl>` — should insert a
field-value pair or update the value of the field in the record associated with key. Also
sets its Time-To-Live starting at timestamp to be ttl. The ttl is the amount of time that this
field-value pair should exist in the database, meaning it will be available during this
interval: `[timestamp, timestamp + ttl)`. This operation should return an empty string.

- `DELETE_AT <key> <field> <timestamp>` — the same as DELETE, but with timestamp of the
operation specified. Should return "true" if the field existed and was successfully deleted
and "false" if the key didn't exist.

- `GET_AT <key> <field> <timestamp>` — the same as GET, but with timestamp of the
operation specified.

- `SCAN_AT <key> <timestamp>` — the same as SCAN, but with timestamp of the operation
specified.

- `SCAN_BY_PREFIX_AT <key> <prefix> <timestamp>` — the same as SCAN_BY_PREFIX, but with
timestamp of the operation specified.


Examples 1:

| Queries | Explanations |
|---------|--------------|
| `["SET_AT_WITH_TTL", "A", "BC", "E", "1", "9"]` | returns `""` <br>database state: `{"A": {"BC": "E"}}` <br>where `{"BC": "E"}` expires at timestamp 10 |
| `["SET_AT_WITH_TTL", "A", "BC", "E", "5", "10"]` | returns `""` <br>database state: `{"A": {"BC": "E"}}` <br>as field "BC" in record "A" already exists, it was overwritten, and `{"BC": "E"}` now expires at timestamp 15 |
| `["SET_AT", "A", "BD", "F", "5"]` | returns `""` <br>database state: `{"A": {"BC": "E", "BD": "F"}}` <br>where `{"BD": "F"}` does not expire |
| `["SCAN_BY_PREFIX_AT", "A", "B", "14"]` | returns `"BC(E), BD(F)"` |
| `["SCAN_BY_PREFIX_AT", "A", "B", "15"]` | returns `"BD(F)"` |

Queries Explanation:

`queries = [
    ["SET_AT_WITH_TTL", "A", "BC", "E", "1", "9"],
    ["SET_AT_WITH_TTL", "A", "BC", "E", "5", "10"],
    ["SET_AT", "A", "BD", "F", "5"],
    ["SCAN_BY_PREFIX_AT", "A", "B", "14"],
    ["SCAN_BY_PREFIX_AT", "A", "B", "15"]
]`

the output should be `["", "", "", "BC(E), BD(F)", "BD(F)"]` 


Examples 2:

| Queries | Explanations |
|---------|--------------|
| `["SET_AT", "A", "B", "C", "1"]` | returns `""` <br>database state: `{"A": {"B": "C"}}` |
| `["SET_AT_WITH_TTL", "X", "Y", "Z", "2", "15"]` | returns `""` <br>database state: `{"X": {"Y": "Z"}, "A": {"B": "C"}}` <br>where `{"Y": "Z"}` expires at timestamp 17 |
| `["GET_AT", "X", "Y", "3"]` | returns `"Z"` |
| `["SET_AT_WITH_TTL", "A", "D", "E", "4", "10"]` | returns `""` <br>database state: `{"X": {"Y": "Z"}, "A": {"D": "E", "B": "C"}}` <br>where `{"D": "E"}` expires at timestamp 14 and `{"Y": "Z"}` expires at timestamp 17 |
| `["SCAN_AT", "A", "13"]` | returns `"B(C), D(E)"` |
| `["SCAN_AT", "X", "16"]` | returns `"Y(Z)"` |
| `["SCAN_AT", "X", "17"]` | returns `""`<br>Note that all fields in record "X" have expired |
| `["DELETE_AT", "X", "Y", "20"]` | returns `"false"`<br>the record "X" was expired at timestamp 17 and can't be deleted |


Queries Explanation:
`queries = [
["SET_AT", "A", "B", "C",
"1"],
["SET_AT_WITH_TTL", "X",
"Y", "Z", "2", "15"],
["GET_AT", "X", "Y", "3"],
["SET_AT_WITH_TTL", "A",
"D", "E", "4", "10"],
["SCAN_AT", "A", "13"],
["SCAN_AT", "X", "16"],
["SCAN_AT", "X", "17"],
["DELETE_AT", "X", "Y",
"20"]
]`

the output should be ["", "", "Z", "", "B(C), D(E)", "Y(Z)", "", "false"].

In [67]:
import copy

class MemDB:
    def __init__(self):
        self.data = {}
        self.data_ts = {}

    def set(self, key, field, value):
        if key not in self.data:
            self.data[key] = {}
        self.data[key][field] = value
        return ""

    def get(self, key, field):
        if key in self.data:
            return self.data[key].get(field, "")
        return ""
    
    def delete(self, key, field):
        if key in self.data and field in self.data[key]:
            del self.data[key][field]
            return "true"
        return "false"
    
    def scan(self, key): 
        if key in self.data:
            res = []
            for field, value in self.data[key].items():
                res.append("%s(%s)" % (field, value))
            return ", ".join(sorted(res))
        return ""

    def scan_by_prefix(self, key, prefix):
        if key in self.data:
            res = []
            for field, value in self.data[key].items():
                if field.startswith(prefix):
                    res.append("%s(%s)" % (field, value))
            return ", ".join(sorted(res))
        return ""
    
    def set_at(self, key, field, value, timestamp:float):
        if key not in self.data_ts:
            self.data_ts[key] = {}

        self.data_ts[key][field] = (value, timestamp, float('inf'))
        return ""

    def set_at_with_ttl(self, key, field, value, timestamp:float, ttl:float):
        if key not in self.data_ts:
            self.data_ts[key] = {}
        
        self.data_ts[key][field] = (value, timestamp, ttl)
        return ""

    def delete_at(self, key, field, timestamp:float):
        if key in self.data_ts and field in self.data_ts[key]:
            _, ts, ttl = self.data_ts[key][field]
            if ts <= timestamp < ts + ttl:
                del self.data_ts[key][field]
                return "true"
        
        return "false"

    def get_at(self, key, field, timestamp:float):
        if key in self.data_ts and field in self.data_ts[key]:
            value, ts, ttl = self.data_ts[key][field]
            if ts <= timestamp < ts + ttl:
                return value
        return ""

    def scan_at(self, key, timestamp:float):
        if key in self.data_ts:
            res = []
            for field, (value, ts, ttl) in self.data_ts[key].items():
                if ts <= timestamp < ts + ttl:
                    res.append("%s(%s)" % (field, value))
            return ", ".join(sorted(res))
        
        return ""

    def scan_by_prefix_at(self, key, prefix, timestamp:float):
        if key in self.data_ts:
            res = []
            for field, (value, ts, ttl) in self.data_ts[key].items():
                if ts <= timestamp < ts + ttl and field.startswith(prefix):
                    res.append("%s(%s)" % (field, value))
            return ", ".join(sorted(res))
        return ""


def solution(queries):
    db = MemDB()
    res = []
    db_ts_status = []
    for query in queries:
        if query[0] == "SET":
            res.append(db.set(query[1], query[2], query[3]))
        elif query[0] == "GET":
            res.append(db.get(query[1], query[2]))
        elif query[0] == "DELETE":
            res.append(db.delete(query[1], query[2]))
        elif query[0] == "SCAN":
            res.append(db.scan(query[1]))
        elif query[0] == "SCAN_BY_PREFIX":
            res.append(db.scan_by_prefix(query[1], query[2]))
        elif query[0] == "SET_AT":
            res.append(db.set_at(query[1], query[2], query[3], float(query[4])))
        elif query[0] == "SET_AT_WITH_TTL":
            res.append(db.set_at_with_ttl(query[1], query[2], query[3], float(query[4]), float(query[5])))
        elif query[0] == "GET_AT":
            res.append(db.get_at(query[1], query[2], float(query[3])))
        elif query[0] == "DELETE_AT":
            res.append(db.delete_at(query[1], query[2], float(query[3])))
        elif query[0] == "SCAN_AT":
            res.append(db.scan_at(query[1], float(query[2])))
        elif query[0] == "SCAN_BY_PREFIX_AT":
            res.append(db.scan_by_prefix_at(query[1], query[2], float(query[3])))

        db_ts_status.append(copy.deepcopy(db.data_ts))

    return res, db_ts_status

queries_1 = [
    ["SET_AT_WITH_TTL", "A", "BC", "E", "1", "9"],
    ["SET_AT_WITH_TTL", "A", "BC", "E", "5", "10"],
    ["SET_AT", "A", "BD", "F", "5"],
    ["SCAN_BY_PREFIX_AT", "A", "B", "14"],
    ["SCAN_BY_PREFIX_AT", "A", "B", "15"]
]

res_1, db_ts_status_1 = solution(queries_1)
print(res_1)
print(db_ts_status_1)

queries_2 = [
    ["SET_AT", "A", "B", "C","1"],
    ["SET_AT_WITH_TTL", "X", "Y", "Z", "2", "15"],
    ["GET_AT", "X", "Y", "3"],
    ["SET_AT_WITH_TTL", "A", "D", "E", "4", "10"],
    ["SCAN_AT", "A", "13"],
    ["SCAN_AT", "X", "16"],
    ["SCAN_AT", "X", "17"],
    ["DELETE_AT", "X", "Y", "20"]
]

res_2, db_ts_status_2 = solution(queries_2)
print(res_2)
print(db_ts_status_2)


['', '', '', 'BC(E), BD(F)', 'BD(F)']
[{'A': {'BC': ('E', 1.0, 9.0)}}, {'A': {'BC': ('E', 5.0, 10.0)}}, {'A': {'BC': ('E', 5.0, 10.0), 'BD': ('F', 5.0, inf)}}, {'A': {'BC': ('E', 5.0, 10.0), 'BD': ('F', 5.0, inf)}}, {'A': {'BC': ('E', 5.0, 10.0), 'BD': ('F', 5.0, inf)}}]
['', '', 'Z', '', 'B(C), D(E)', 'Y(Z)', '', 'false']
[{'A': {'B': ('C', 1.0, inf)}}, {'A': {'B': ('C', 1.0, inf)}, 'X': {'Y': ('Z', 2.0, 15.0)}}, {'A': {'B': ('C', 1.0, inf)}, 'X': {'Y': ('Z', 2.0, 15.0)}}, {'A': {'B': ('C', 1.0, inf), 'D': ('E', 4.0, 10.0)}, 'X': {'Y': ('Z', 2.0, 15.0)}}, {'A': {'B': ('C', 1.0, inf), 'D': ('E', 4.0, 10.0)}, 'X': {'Y': ('Z', 2.0, 15.0)}}, {'A': {'B': ('C', 1.0, inf), 'D': ('E', 4.0, 10.0)}, 'X': {'Y': ('Z', 2.0, 15.0)}}, {'A': {'B': ('C', 1.0, inf), 'D': ('E', 4.0, 10.0)}, 'X': {'Y': ('Z', 2.0, 15.0)}}, {'A': {'B': ('C', 1.0, inf), 'D': ('E', 4.0, 10.0)}, 'X': {'Y': ('Z', 2.0, 15.0)}}]


Level 4

The database should be backed up from time to time. Introduce operations to support backing up
and restoring the database state based on timestamps. When restoring, ttl expiration times should
be recalculated accordingly.

- `BACKUP <timestamp>` — should save the database state at the specified timestamp, including
the remaining ttl for all records and fields. Remaining ttl is the difference between their
initial ttl and their current lifespan (the duration between the timestamp of this operation
and their initial timestamp). Returns a string representing the number of non-empty
non-expired records in the database.

- `RESTORE <timestamp> <timestampToRestore>` — should restore the database from the latest
backup before or at timestampToRestore. It's guaranteed that a backup before or at
timestampToRestore will exist. Expiration times for restored records and fields should be
recalculated according to the timestamp of this operation - since the database timeline
always flows forward, restored records and fields should expire after the timestamp of this
operation, depending on their remaining ttls at backup. This operation should return an
empty string.

Examples

| Query | Result/Database State |
|-------|---------------------|
| `["SET_AT_WITH_TTL", "A", "B", "C", "1", "10"]` | `""` ; Database: `{"A": {"B": "C"}}` with lifespan `[1, 11)` |
| `["BACKUP", "3"]` | `"1"` ; Saves database state |
| `["SET_AT", "A", "D", "E", "4"]` | `""` ; Database: `{"A": {"D": "E", "B": "C"}}` |
| `["BACKUP", "5"]` | `"1"` ; Saves database state |
| `["DELETE_AT", "A", "B", "8"]` | `"true"` ; Database: `{"A": {"D": "E"}}` |
| `["BACKUP", "9"]` | `"1"` ; Saves database state |
| `["RESTORE", "10", "7"]` | `""` ; Restores to state: `{"A": {"D": "E", "B": "C"}}` with `{"B": "C"}` expiring at t=16 |
| `["BACKUP", "11"]` | `"1"` ; Saves database state |
| `["SCAN_AT", "A", "15"]` | `"B(C), D(E)"` |
| `["SCAN_AT", "A", "16"]` | `"D(E)"` |


Query Explanation:

`queries = [
    ["SET_AT_WITH_TTL", "A", "B", "C", "1", "10"],
    ["BACKUP", "3"],
    ["SET_AT", "A", "D", "E", "4"],
    ["BACKUP", "5"],
    ["DELETE_AT", "A", "B", "8"],
    ["BACKUP", "9"],
    ["RESTORE", "10", "7"],
    ["BACKUP", "11"],
    ["SCAN_AT", "A", "15"],
    ["SCAN_AT", "A", "16"]
]`

The output should be `["", "1", "", "1", "true", "1", "", "1", "B(C), D(E)", "D(E)"]`.

More queries examples:

`queries = [
    ["BACKUP","160000000"],
    ["RESTORE","160000001","160000000"],
    ["SET_AT_WITH_TTL","key","field","str","160000100","200"],
    ["BACKUP","160000200"],
    ["RESTORE","160000250","160000000"],
    ["GET_AT","key","field","160000300"],
    ["BACKUP","160000350"],
    ["RESTORE","160000400","160000200"],
    ["GET_AT","key","field","160000450"],
    ["GET_AT","key","field","160000500"]
]`

`queries = [
    ["SET_AT","foo","bar","baz","160000100"],
    ["SET_AT_WITH_TTL","key","key","value","160000120","1880"],
    ["SET_AT_WITH_TTL","key","key","value","160000170","680"],
    ["SET_AT_WITH_TTL","bar","baz","foo","160000200","100"],
    ["BACKUP","160000250"],
    ["BACKUP","160000270"],
    ["SET_AT_WITH_TTL","boo","text1","text2","160000300","900"],
    ["BACKUP","160000850"],
    ["GET_AT","foo","bar","160000900"],
    ["SCAN_BY_PREFIX_AT","key","k","160000920"],
    ["DELETE_AT","foo","bar","160000950"],
    ["DELETE_AT","key","key","160000960"],
    ["RESTORE","160000970","160000250"],
    ["SCAN_AT","key","160000980"],
    ["SCAN_AT","key","160001021"],
    ["RESTORE","160001030","160000850"],
    ["SCAN_AT","key","160001040"],
    ["SCAN_AT","boo","160001041"],
    ["SCAN_AT","bar","160001042"]
]`

`queries = [
    ["DELETE_AT","key","key","160000010"],
    ["DELETE_AT","key","key2","160000020"],
    ["SCAN_BY_PREFIX_AT","key","key","160000025"],
    ["GET_AT","A","B","160000030"],
    ["GET_AT","key","key","160000040"],
    ["SCAN_BY_PREFIX_AT","key","key","160000050"],
    ["DELETE_AT","key","key","160000052"],
    ["BACKUP","160000055"],
    ["SET_AT_WITH_TTL","key","key","aaaaa","160000060","1940"],
    ["SET_AT_WITH_TTL","foo","bar","baz","160000070","101"],
    ["DELETE_AT","key","bar","160000080"],
    ["DELETE_AT","key","key2","160000090"],
    ["BACKUP","160000100"],
    ["SET_AT_WITH_TTL","key","key","otherValue","160000120","20"],
    ["RESTORE","160000130","160000099"],
    ["SCAN_BY_PREFIX_AT","key","key","160000150"],
    ["RESTORE","160000160","160000100"],
    ["SCAN_BY_PREFIX_AT","key","k","160000200"],
    ["SCAN_BY_PREFIX_AT","key","k","160000201"],
    ["RESTORE","160000250","160000110"],
    ["SCAN_AT","key","160000270"],
    ["SCAN_BY_PREFIX_AT","key","key","160000350"]
]`

In [82]:
import copy

class MemDB:
    def __init__(self):
        self.data = {}
        self.data_ts = {}
        self.backup_data = {}

    def set(self, key, field, value):
        if key not in self.data:
            self.data[key] = {}
        self.data[key][field] = value
        return ""

    def get(self, key, field):
        if key in self.data:
            return self.data[key].get(field, "")
        return ""
    
    def delete(self, key, field):
        if key in self.data and field in self.data[key]:
            del self.data[key][field]
            return "true"
        return "false"
    
    def scan(self, key): 
        if key in self.data:
            res = []
            for field, value in self.data[key].items():
                res.append("%s(%s)" % (field, value))
            return ", ".join(sorted(res))
        return ""

    def scan_by_prefix(self, key, prefix):
        if key in self.data:
            res = []
            for field, value in self.data[key].items():
                if field.startswith(prefix):
                    res.append("%s(%s)" % (field, value))
            return ", ".join(sorted(res))
        return ""
    
    def set_at(self, key, field, value, timestamp:float):
        if key not in self.data_ts:
            self.data_ts[key] = {}

        self.data_ts[key][field] = (value, timestamp, float('inf'))
        return ""

    def set_at_with_ttl(self, key, field, value, timestamp:float, ttl:float):
        if key not in self.data_ts:
            self.data_ts[key] = {}
        
        self.data_ts[key][field] = (value, timestamp, ttl)
        return ""

    def delete_at(self, key, field, timestamp:float):
        if key in self.data_ts and field in self.data_ts[key]:
            _, ts, ttl = self.data_ts[key][field]
            if ts <= timestamp < ts + ttl:
                del self.data_ts[key][field]
                return "true"
        
        return "false"

    def get_at(self, key, field, timestamp:float):
        if key in self.data_ts and field in self.data_ts[key]:
            value, ts, ttl = self.data_ts[key][field]
            if ts <= timestamp < ts + ttl:
                return value
        return ""

    def scan_at(self, key, timestamp:float):
        if key in self.data_ts:
            res = []
            for field, (value, ts, ttl) in self.data_ts[key].items():
                if ts <= timestamp < ts + ttl:
                    res.append("%s(%s)" % (field, value))
            return ", ".join(sorted(res))
        
        return ""

    def scan_by_prefix_at(self, key, prefix, timestamp:float):
        if key in self.data_ts:
            res = []
            for field, (value, ts, ttl) in self.data_ts[key].items():
                if ts <= timestamp < ts + ttl and field.startswith(prefix):
                    res.append("%s(%s)" % (field, value))
            return ", ".join(sorted(res))
        return ""
    
    def backup(self, timestamp:float):
        count = 0 # number of non-empty key records
        for key in self.data_ts.keys():
            non_empty_flag = False
            for field, (value, ts, ttl) in self.data_ts[key].items():
                if ttl != float('inf'):
                    lifespan = ts + ttl - timestamp
                    if lifespan > 0:
                        # update the timestamp, ttl to current timestamp and current lifespan
                        self.data_ts[key][field] = (value, timestamp, lifespan)
                        non_empty_flag = True
                else:
                    non_empty_flag = True
    
            if non_empty_flag:
                count += 1

        self.backup_data[timestamp] = copy.deepcopy(self.data_ts)
        return str(count)
  

    def restore(self, timestamp:float, timestampToRestore:float):
        backup_timestamps = sorted(self.backup_data.keys())

        backup_ts_to_restore = backup_timestamps[0]        
        for ts in backup_timestamps:
            if ts <= timestampToRestore:
                backup_ts_to_restore = ts
            else:
                break
        
        self.data_ts = copy.deepcopy(self.backup_data[backup_ts_to_restore])

        for key in self.data_ts.keys():
            for field, (value, ts, ttl) in self.data_ts[key].items():
                self.data_ts[key][field] = (value, timestamp, ttl)
                
        return ""


def solution(queries):
    db = MemDB()
    res = []
    db_ts_status = []
    db_backup_status = []
    for query in queries:
        if query[0] == "SET":
            res.append(db.set(query[1], query[2], query[3]))
        elif query[0] == "GET":
            res.append(db.get(query[1], query[2]))
        elif query[0] == "DELETE":
            res.append(db.delete(query[1], query[2]))
        elif query[0] == "SCAN":
            res.append(db.scan(query[1]))
        elif query[0] == "SCAN_BY_PREFIX":
            res.append(db.scan_by_prefix(query[1], query[2]))
        elif query[0] == "SET_AT":
            res.append(db.set_at(query[1], query[2], query[3], float(query[4])))
        elif query[0] == "SET_AT_WITH_TTL":
            res.append(db.set_at_with_ttl(query[1], query[2], query[3], float(query[4]), float(query[5])))
        elif query[0] == "GET_AT":
            res.append(db.get_at(query[1], query[2], float(query[3])))
        elif query[0] == "DELETE_AT":
            res.append(db.delete_at(query[1], query[2], float(query[3])))
        elif query[0] == "SCAN_AT":
            res.append(db.scan_at(query[1], float(query[2])))
        elif query[0] == "SCAN_BY_PREFIX_AT":
            res.append(db.scan_by_prefix_at(query[1], query[2], float(query[3])))
        elif query[0] == "BACKUP":
            res.append(db.backup(float(query[1])))
        elif query[0] == "RESTORE":
            res.append(db.restore(float(query[1]), float(query[2])))

        db_ts_status.append(copy.deepcopy(db.data_ts))
        db_backup_status.append(copy.deepcopy(db.backup_data))

    return res, db_ts_status, db_backup_status

queries_1 = [
    ["SET_AT_WITH_TTL", "A", "B", "C", "1", "10"],
    ["BACKUP", "3"],
    ["SET_AT", "A", "D", "E", "4"],
    ["BACKUP", "5"],
    ["DELETE_AT", "A", "B", "8"],
    ["BACKUP", "9"],
    ["RESTORE", "10", "7"],
    ["BACKUP", "11"],
    ["SCAN_AT", "A", "15"],
    ["SCAN_AT", "A", "16"]
]

res_1, db_ts_status_1, db_backup_status_1 = solution(queries_1)

print(res_1)
for i, (return_value, data_ts, backup_data) in enumerate(zip(res_1, db_ts_status_1, db_backup_status_1)):
    print(f"Query {i+1}: {queries_1[i]}")
    print(f"return value: {return_value}")
    print(f"data_ts: {data_ts}")
    print(f"backup_data: {backup_data}")

['', '1', '', '1', 'true', '1', '', '1', 'B(C), D(E)', 'D(E)']
Query 1: ['SET_AT_WITH_TTL', 'A', 'B', 'C', '1', '10']
return value: 
data_ts: {'A': {'B': ('C', 1.0, 10.0)}}
backup_data: {}
Query 2: ['BACKUP', '3']
return value: 1
data_ts: {'A': {'B': ('C', 3.0, 8.0)}}
backup_data: {3.0: {'A': {'B': ('C', 3.0, 8.0)}}}
Query 3: ['SET_AT', 'A', 'D', 'E', '4']
return value: 
data_ts: {'A': {'B': ('C', 3.0, 8.0), 'D': ('E', 4.0, inf)}}
backup_data: {3.0: {'A': {'B': ('C', 3.0, 8.0)}}}
Query 4: ['BACKUP', '5']
return value: 1
data_ts: {'A': {'B': ('C', 5.0, 6.0), 'D': ('E', 4.0, inf)}}
backup_data: {3.0: {'A': {'B': ('C', 3.0, 8.0)}}, 5.0: {'A': {'B': ('C', 5.0, 6.0), 'D': ('E', 4.0, inf)}}}
Query 5: ['DELETE_AT', 'A', 'B', '8']
return value: true
data_ts: {'A': {'D': ('E', 4.0, inf)}}
backup_data: {3.0: {'A': {'B': ('C', 3.0, 8.0)}}, 5.0: {'A': {'B': ('C', 5.0, 6.0), 'D': ('E', 4.0, inf)}}}
Query 6: ['BACKUP', '9']
return value: 1
data_ts: {'A': {'D': ('E', 4.0, inf)}}
backup_data: {3.0: {'